In [ ]:
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import pandas as pd
from transformers import AutoTokenizer, TFAutoModel
import tensorflow as tf
import numpy as np
from tqdm import tqdm

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/datasets/snapp.csv")
df.head()

,comment_cleaned,label
0,واقعا حیف وقت بنویسم سرویس دهیتون افتضاح,1
1,قرار ساعته برسه نیم ساعت زودتر موقع ببین چقدر ...,0
2,قیمت مدل اصلا کیفیتش سازگاری نداره ظاهر فریبند...,1
3,درست اندازه کیفیت امیداورم کیفیتتون باشه مشتری...,0
4,شیرینی وانیلی مدل,0


In [ ]:
# لود tokenizer و مدل ParsBERT
tokenizer = AutoTokenizer.from_pretrained("HooshvareLab/bert-base-parsbert-uncased")
bert_model = TFAutoModel.from_pretrained("HooshvareLab/bert-base-parsbert-uncased")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/434 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.22M [00:00<?, ?B/s]

tf_model.h5:   0%|          | 0.00/963M [00:00<?, ?B/s]

Some layers from the model checkpoint at HooshvareLab/bert-base-parsbert-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at HooshvareLab/bert-base-parsbert-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
texts = df['comment_cleaned'].astype(str)
labels = df['label']


# تبدیل جملات به embedding با ParsBERT (استفاده از [CLS] token)
def get_bert_embeddings(texts, tokenizer, model, max_len=64):
    embeddings = []
    for text in tqdm(texts):
        inputs = tokenizer(text, return_tensors='tf', max_length=max_len, truncation=True, padding='max_length')
        outputs = model(inputs)[0]  # shape: (1, seq_len, hidden)
        cls_vector = outputs[:, 0, :]  # فقط بردار CLS
        embeddings.append(cls_vector.numpy()[0])
    return np.array(embeddings)

In [ ]:
# ساخت embedding برای تمام داده‌ها
X = get_bert_embeddings(texts, tokenizer, bert_model)
y = np.array(labels)

 86%|████████▌ | 59983/70000 [3:31:12<45:32,  3.67it/s]

In [ ]:
import pickle

with open('data.pkl', 'wb') as f:
    pickle.dump((X, y), f)